In [ ]:
dirname = 'logs/thumbnails1'

In [ ]:
from duckietown_utils import locate_files
locate_files(dirname, 'image-*.jpg')

In [ ]:
from duckietown_utils import image_cv_from_jpg_fn, rgb_from_bgr

def load_images_rgb(dirname):
    """ Load all the images from dirname in RGB format """
    
    # look for files
    files = locate_files(dirname, 'image-*jpg')
    
    # sort them in sequence
    files = sorted(files)
    
    # load all of them using image_cv_from_jpg_fn() that loads a JPG image 
    f = ErniComposition([image_cv_from_jpg_fn, rgb_from_bgr])
    
    images_rgb = map(f, files)
    
    return images
    
class ErniComposition():
    def __init__(self, fs):
        self.fs = fs

    def __call__(self, x):
        for f in self.fs:
            x = f(x)
        return x
    # option 0
#    images3 = []
#     for image in images:
#         images3.append(rgb_from_bgr(image))

#     #     
#     images2 = [ rgb_from_bgr(x) for x in images]

#     images2 = map_ac(rgb_from_bgr, images)
    
#     images_back = map_ac(bgr_from_rgb, images2)
    
#     def map_ac(f, sequence):
#         sequence2 = []
#         for x in sequence:
#             x2 = f(x)
#             sequence2.append(x2)
#         return sequence2
    
    


In [ ]:
images = load_images_rgb(dirname)

In [ ]:
print('Loaded %d images; resolution is %s' % (len(images), images[0].shape))

In [ ]:
I = images[200]

from matplotlib import pyplot
pyplot.imshow(I)
pyplot.show()



In [ ]:
from matplotlib import pyplot
import numpy as np

i = 0

A = images[i ] # uint8
B = images[i + 1]

Af = A.astype('float')
Bf = B.astype('float')

diff = np.abs(Af - Bf)
diff_uint = diff.astype('uint8')

pyplot.imshow(diff_uint);
pyplot.show()

In [ ]:
def get_similarity_matrix(images, phi, distance):
    n = len(images)    
    descriptors = map(phi, images)
    
    A = np.zeros((n,n))
    interval = n / 10
    for i in range(n):
        if i % interval == 0:
            print('%d/%d' % (i, n))
        for j in range(n):
            if i <= j:
                d = distance(descriptors[i], descriptors[j])
                A[i,j] = d
                A[j,i] = d
    return A

In [ ]:

from duckietown_utils import d8_image_zoom_linear

def phi_resize(x):
    x = d8_image_zoom_linear(x, 1/16.0)
    return x.astype('float32')

def distance_L1(a,b):
    return np.mean(np.fabs(a-b))


In [ ]:
A_L1 = get_similarity_matrix(images, phi_resize, distance_L1)

In [ ]:
pyplot.figure()
pyplot.imshow(A)
pyplot.show()

In [ ]:
pyplot.figure()
pyplot.plot(A[100,:])
pyplot.show()